# Heat equation
We are solving the heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0.$$

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from math import pi
from netgen.geom2d import SplineGeometry

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [ ]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.05))
Draw(mesh)
fes = H1(mesh, order=1, dirichlet="bottom|right|left|top")

u,v = fes.TnT() # TnT : Trial and Test function

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix seperately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

$$
  M \frac{u^{n+1}-u^{n}}{\Delta t} + A(\nu u^{n+1}+ (1-\nu)u^{n}) = \nu f^{n+1}+ (1-\nu)f^{n}
$$

In [ ]:
a = BilinearForm(fes)
a += (0.005*grad(u)*grad(v)) * dx
a.Assemble()

m = BilinearForm(fes)
m += (u*v) * dx
m.Assemble()

## Implicit vs. Explicit Euler for $f=0$

### Explicit Euler method, i.e $\nu=0$
$$
  M u^{n+1} = M u^n + \Delta t(-Au^n )
$$

In [ ]:
dt = 0.01 # try 0.05 to break explicit euler

In [ ]:
invm = m.mat.Inverse(freedofs=fes.FreeDofs())

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,'u',autoscale=False,min=-1,max=1)
gfut = GridFunction(gfu.space,multidim=0)
gfut.AddMultiDimComponent(gfu.vec)

In [ ]:
time=0
res = gfu.vec.CreateVector()
while time < 20:
    res.data = m.mat * gfu.vec - dt * a.mat * gfu.vec
    gfu.vec.data = invm * res
    time += dt
    print("\r",time,end="")
    if round(time,2) % 1 == 0:
        gfut.AddMultiDimComponent(gfu.vec)
print("")


In [ ]:
Draw(gfut, mesh, interpolate_multidim=True, animate=True)

### Implicit Euler method, i.e $\nu=1$
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n
$$

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [ ]:
mstar = m.mat.CreateMatrix()
#print(mstar)

To access the entries we use the vector of nonzero-entries:

In [ ]:
mstar.AsVector()
#print(mstar.AsVector())

In [ ]:
print(mstar.nze)
print(len(mstar.AsVector()))

Using the vector we can build the linear combination of the a and the m matrix:

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,'u',autoscale=False,min=-1,max=1)

In [ ]:
time = 0
res = gfu.vec.CreateVector()
while time < 30:
    res.data = m.mat * gfu.vec
    gfu.vec.data = invmstar * res
    time += dt
    print("\r",time,end="")
    Redraw(blocking=True)

## Time constant rhs.
Implicit Euler method, i.e $\nu=1$ in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = \Delta t (-A u^n + f^{n+1}).
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [ ]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

In [ ]:
time = 0 
res = gfu.vec.CreateVector()
while time < 30:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    time += dt
    print("\r",time,end="")
    Redraw(blocking=True)

### Alternative version with iterative solvers

* For a factorization of $M^\ast$ (${M^\ast}^{-1}$) we require a sparse matrix $M^\ast$ 
* To store $M^\ast$ as a sparse matrix requires new storage (and same memory layout of $A$ and $M$)
* For iterative solvers we only require the matrix (and preconditioner) applications
* `mstar = m.mat + dt * a.mat` has no storage but matrix-vector multiplications

iterative solver version (with Jacobi preconditining):

In [ ]:
mstar_alt = m.mat + dt * a.mat
premstar_alt = m.mat.CreateSmoother() + dt * a.mat.CreateSmoother()
invmstar_alt = CGSolver(mstar_alt,premstar_alt,printrates=True)

print(premstar_alt)

## Supplementary 1: time-dependent r.h.s. data
Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
Draw(gausspt,mesh,"ft")
time = 0.0
while time < 10:
    t.Set(time)
    Redraw(blocking=True)
    time += 1e-5

Accordingly we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += SymbolicLFI(gausspt*v)
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
Draw(gfu,mesh,"u")

In [ ]:
time = 0 
res = gfu.vec.CreateVector()
while time < 30:
    t.Set(time)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    time += dt
    print("\r",time,end="")
    Redraw(blocking=True)

## Supplementary 2: Time dependent boundary conditions

* $u|_{\partial \Omega} = u_D(t)$, $f=0$
* implicit Euler time stepping method, non-incremental form:

  $$
    M^\ast u^{n+1} = (M + \Delta t A) u^{n+1} = M u^n
  $$  
  
* Homogenize w.r.t. to boundary conditions, i.e. we split 

  $$ u^{n+1} = u^{n+1}_0 + u^{n+1}_D $$ 
  
  where $u^{n+1}_D$ is a (discrete) function with correct boundary condition:  
  
$$
  {M^\ast} u^{n+1}_0 = M u^n - {M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

In [ ]:
time = 0 
res = gfu.vec.CreateVector()
while time < 50:
    t.Set(time)
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    time += dt
    print("\r",time,end="")
    Redraw(blocking=True)